In [1]:
import pandas as pd
import requests
import json
import datetime

In [2]:
gare = "Montpellier Saint-Roch"

# Max 250
nb_trains = 20

In [3]:
token = 'e7b7fedd-71d0-48c6-8cc7-749e22ba8e80'

## Dictionnaire

In [4]:
req_gare = requests.get("https://ressources.data.sncf.com/api/records/1.0/search/?dataset=referentiel-gares-voyageurs&q=&rows=3220&sort=gare_alias_libelle_noncontraint&facet=departement_libellemin&facet=segmentdrg_libelle&facet=gare_agencegc_libelle&facet=gare_regionsncf_libelle&facet=gare_ug_libelle")
doc_gare = json.loads(req_gare.text)
row_gare = len(doc_gare['records'])
print(f'Nombre de lignes : {row_gare}')

Nombre de lignes : 3220


In [5]:
df_dic = pd.DataFrame(doc_gare['records'])
df_dic = pd.DataFrame(list(df_dic['fields']))
df_dic = df_dic[['alias_libelle_noncontraint','uic_code']]

In [6]:
def removezero(string):
    return 'SNCF:' + str(string)[2:]

df_dic['uic_code'] = df_dic['uic_code'].apply(removezero)

In [7]:
df_dic.convert_dtypes()
dic_gare = df_dic.set_index('alias_libelle_noncontraint').T.to_dict('list')

C:\Users\hp\AppData\Local\Temp\ipykernel_4576\3118890846.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dic_gare = df_dic.set_index('alias_libelle_noncontraint').T.to_dict('list')


## Début

In [8]:
link = 'https://api.sncf.com/v1/coverage/sncf/stop_areas/stop_area:' + dic_gare[gare][0] + '/arrivals?count=' + str(nb_trains)
req = requests.get(link,auth=(token, ''))
#print(link)

In [9]:
doc = json.loads(req.text)
row = len(doc['arrivals'])
print(f'Nombre de lignes : {row}')

Nombre de lignes : 20


In [10]:
df = pd.DataFrame(doc['arrivals'])
df_gare = pd.DataFrame(list(df['display_informations']))
df_heure = pd.DataFrame(list(df['stop_date_time']))
df_id = pd.DataFrame(list(df['links']))
df_id = pd.DataFrame(list(df_id[1]))

In [11]:
def del_day(string):
    return string[9:]

df_heure['arrival_date_time'] = df_heure['arrival_date_time'].apply(del_day)
df_heure['base_arrival_date_time'] = df_heure['base_arrival_date_time'].apply(del_day)

In [12]:
def conv_min(string):
    return int(string[0:2])*60 + int(string[2:4])

df_heure['retard'] = df_heure['arrival_date_time'].apply(conv_min) - df_heure['base_arrival_date_time'].apply(conv_min)

In [13]:
def str_tps(str):
    return datetime.datetime.strptime(str, '%H%M%S').time()

df_heure['heure'] = df_heure['arrival_date_time'].apply(str_tps)
df_heure['old_heure'] = df_heure['base_arrival_date_time'].apply(str_tps)

In [14]:
def del_par(string):
    index = string.find("(")
    return string[:index]

df_gare['direction'] = df_gare['direction'].apply(del_par)

In [15]:
df_gare = df_gare[['direction','network','trip_short_name']]
df_gare.rename(columns = {'direction':'Destination'}, inplace = True)
df_gare.rename(columns = {'network':'Train'}, inplace = True)
df_gare.rename(columns = {'trip_short_name':'Numéro'}, inplace = True)

df_gare['Arrivée (réelle)'] = df_heure['heure']
df_gare['Arrivée (prévue)'] = df_heure['old_heure']
df_gare['Retard (min)'] = df_heure['retard']
df_gare['id'] = df_id['id']

In [16]:
def get_name(string):
    string = string[10:]
    index_fin = string.find("', 'links'")
    return string[:index_fin]

arrets = []
causes = []

for i in df_gare.index:
    id = df_gare.iloc[i]['id']
    
    if 'RealTime' in id:
        index_id = id.index("RealTime")
        id = id[:index_id-1]

    link_voyage = 'https://api.sncf.com/v1/coverage/sncf/vehicle_journeys/' + id
    req_arret = requests.get(link_voyage ,auth=(token, ''))
    doc_voyage = json.loads(req_arret.text)

    df_arret = pd.DataFrame(doc_voyage['vehicle_journeys'])
    df_arret = pd.DataFrame(list(df_arret['stop_times']))
    df_arret = df_arret.T
    df_arret = pd.DataFrame(list(df_arret[0]))
    df_arret['stop_point'] = df_arret['stop_point'].astype('str').apply(get_name)

    if df_gare.iloc[i]['Retard (min)'] != 0:
        df_retard = pd.DataFrame(doc_voyage['disruptions'])
        if 'messages' in df_retard:
            df_retard = pd.DataFrame(list(df_retard['messages'][0]))
            causes.append(df_retard.iloc[0]['text'])
        else:
            causes.append("Retard non expliqué")
    else:
        causes.append("")

    liste_arrets = list(df_arret['stop_point'])

    arrets.append(liste_arrets)

In [17]:
df_gare['Cause'] = causes
df_gare['Arrêts'] = arrets
df_gare = df_gare.drop(['id'], axis=1)

In [18]:
df_gare

,Destination,Train,Numéro,Arrivée (réelle),Arrivée (prévue),Retard (min),Cause,Arrêts
0,Perpignan,TER,876423,16:48:00,16:48:00,0,,"[Avignon Centre, Tarascon, Beaucaire, Nîmes Po..."
1,Marseille - Saint-Charles,Intercités,4661,16:52:00,16:52:00,0,,"[Bordeaux Saint-Jean, Agen, Montauban Ville Bo..."
2,Bordeaux Saint-Jean,Intercités,4764,17:18:00,17:03:00,15,Régulation du trafic,"[Marseille - Saint-Charles, Nîmes Centre, Mont..."
3,Avignon Centre,TER,876422,17:19:00,17:09:00,10,Défaillance de matériel,"[Perpignan, Rivesaltes, Salses, Leucate la Fra..."
4,Sète,TER,876361,17:20:30,17:20:30,0,,"[Lunel, Valergues - Lansargues, Baillargues, S..."
5,Marseille - Saint-Charles,TER,876546,17:34:00,17:34:00,0,,"[Narbonne, Béziers, Agde, Sète, Frontignan, Mo..."
6,Paris - Gare de Lyon - Hall 1 & 2,TGV INOUI,9704,17:43:00,17:43:00,0,,"[Barcelona Sants, Girona/Gerona, Figueres - Vi..."
7,Lyon Part Dieu,TGV INOUI,6873,17:55:00,17:55:00,0,,"[Toulouse Matabiau, Carcassonne, Narbonne, Béz..."
8,Avignon Centre,TER,876424,18:09:00,18:09:00,0,,"[Narbonne, Béziers, Agde, Sète, Frontignan, Mo..."
9,Montpellier Saint-Roch,OUIGO,7875,18:10:00,18:10:00,0,,"[Paris - Gare de Lyon - Hall 1 & 2, Lyon Saint..."
